### Estimation of Current Stations

Because the data is only through December 2016, this is an approximation of the number of current stations and their locations.

#### Create a CSV file with the four columns we need (start station name, start station latitude, start station longitude) from the file containing data from December 2016

In [ ]:
import pandas as pd
import csv

path="/Users/hildavargas/Desktop/Springboard_Mini_Projects/Capstone_Project/CitiBike_Data/2016-12-citibike-tripdata.csv"

csv_r=reader(open(path, "r"), delimiter=",")

stations=[]

next(csv_r)

for row in csv_r:
# Append station ID, station name, latitude, longitude
    stations.append((row[3], row[4], row[5], row[6]))

# Remove duplicates
current_stations=set(stations)

with open("/Users/hildavargas/Desktop/Springboard_Mini_Projects/Capstone_Project/CB_real_metrics/StationsAsofDec2016.csv", "w") as out:
    csv_o=csv.writer(out, delimiter=',')
        
    fieldnames=['Start Station ID','Start Station Name', 'Start Station Latitude', 'Start Station Longitude']
        
    csv_o.writerow(fieldnames)
    
    for row in current_stations:
        csv_o.writerow(row)

In [ ]:
import pandas as pd
# Read new CSV as dataframe
current_stationsdf=pd.read_csv("/Users/hildavargas/Desktop/Springboard_Mini_Projects/Capstone_Project/CB_real_metrics/StationsAsofDec2016.csv")
current_stationsdf.head()

,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude
0,2004,6 Ave & Broome St,40.724399,-74.004704
1,3331,Riverside Dr & W 104 St,40.801343,-73.971146
2,522,E 51 St & Lexington Ave,40.757148,-73.972078
3,418,Front St & Gold St,40.702240,-73.982578
4,467,Dean St & 4 Ave,40.683125,-73.978951


#### Create an interactive map of the December 2016 stations with Folium

In [ ]:
import folium

nyc_map=folium.Map(location=[40.756, -73.982], zoom_start=11.9, min_zoom=10, tiles='cartodbdark_matter')
for row in current_stationsdf.itertuples():
    folium.RegularPolygonMarker([row[3], row[4]], popup=row[2], radius=4).add_to(nyc_map)
    
nyc_map